<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/generateCode-codeLlama-myAgent-gsm8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install langchain
!pip install tiktoken
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-community
!pip install sentence_transformers
!pip install peft
# !pip install bitsandbytes
!pip install -U bitsandbytes
!pip install --upgrade torch
!pip install transformers
!pip install huggingface_hub
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━

In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from datasets import load_dataset
import csv

from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Retrieve agent

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

# 모델 경로
embedding_model_path = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
embedding_model = SentenceTransformer(embedding_model_path, device=device)

embedding_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [6]:
import faiss

## FAISS index
index = faiss.read_index("/content/drive/MyDrive/promptGenerate/question_faiss_index-sentence-gsm8k.faiss")

rag_dataset = load_dataset("openai/gsm8k", 'socratic')
rag_dataset

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.68M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/487k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [7]:
import numpy as np

def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

# 유사도 검색 함수
def RAG_AGENT(query_sentence, k=1):
    query_embedding = embedding_model.encode([query_sentence])
    query_embedding_normalized = normalize(query_embedding)

    # FAISS 인덱스에서 가장 유사한 벡터 k개 검색
    distances, indices = index.search(np.array([query_embedding_normalized[0]], dtype='float32'), k)

    # 검색된 인덱스와 유사도를 반환
    similar_questions = []
    for i in range(k):
        question = rag_dataset['train']['question'][indices[0][i]]
        snippet = rag_dataset['train']['answer'][indices[0][i]]
        distance = distances[0][i]

        # print(f"원래 질문: {query_sentence}, 문서 id: {indices[0][i]}, 유사도: {distances[0][i]}, 질문: {question}")
        # similar_questions.append((question, snippet, distance))

    return question, snippet, distance

# Planning Agent

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

tokenizer = AutoTokenizer.from_pretrained("donghuna/llama-plan-1B")
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", return_dict=True)
peft_model_id = "donghuna/llama-plan-1B"
plan_model = PeftModel.from_pretrained(base_model, peft_model_id).to("cuda")
plan_model.eval()


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:500: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

In [9]:


def generate_and_tokenize_planning_prompt(original_question, similar_question, answer, confidence):
    full_prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert reasoning assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Your task is to create a clear and step-by-step plan to solve a math problem.
The plan should guide someone to write Python code that solves the problem effectively.
Use the problem-solving steps and logical reasoning based on the following explanation to generate the plan.

Problem:
{original_question}

Retrieved Problem (similar question):
{similar_question}

Retrieved answer (related to the retrieved question):
{answer}

Similarity score (between original problem and retrieved question):
{confidence}

Generate a plan:
- Create a detailed, step-by-step plan for solving the problem.
- Ensure each step logically follows the explanation above.
- Focus on breaking down the solution into clear, simple steps that can be implemented in Python code.

Output only the plan in a numbered list format.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
    return full_prompt

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

def PLANNING_AGENT(original_question, similar_question, answer, confidence):
    prompt = generate_and_tokenize_planning_prompt(original_question, similar_question, answer, confidence)
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    # 입력 토큰의 길이 계산
    input_length = inputs['input_ids'].shape[1]

    # 입력 프롬프트 토큰 수에 128을 더한 max_length 설정
    max_length = input_length + 256

    with torch.no_grad():
        output_ids = plan_model.generate(
            **inputs,
            max_length=max_length,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.2
        )

        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

        # print(generated_text)
        generated_plan = ""

        keyword = "<|start_header_id|>assistant<|end_header_id|>"
        if keyword in generated_text:
            generated_plan = generated_text.split(keyword)[1].strip()

        keyword = "<|eot_id|>"
        if keyword in generated_plan:
            generated_plan = generated_plan.split(keyword)[0].strip()

        # if not generated_plan:
        #     continue

        # print(generated_plan)
        return generated_plan

# Code generator

In [10]:
# code_model = "codellama/CodeLlama-7b-hf"
code_model = "codellama/CodeLlama-7b-Instruct-hf"
# code_model = "meta-llama/Llama-3.2-1B-Instruct"

code_tokenizer = AutoTokenizer.from_pretrained(code_model, truncation=True)
code_pipeline = transformers.pipeline(
    "text-generation",
    model=code_model,
    tokenizer=code_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
# dataset = load_dataset("openai_humaneval")
# dataset = load_dataset("donghuna/humanEval_with_plan")
# dataset

dataset = load_dataset("openai/gsm8k", 'main')
dataset

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [12]:
import textwrap

# system_prompt = """You are an AI specialized in generating Python code. Only produce Python code that directly corresponds to the provided comments or function descriptions without any additional explanations or extraneous text."""
# system_prompt = "You are an AI assistant that generates Python code based on provided instructions."


def generate_and_tokenize_prompt_for_generate_code(original_question, similar_question, snippet, confidence, plan):
    # plan = f'"""{plan}"""'
    # plan = textwrap.indent(plan, '    ')
    full_prompt = f"""
[INST] <<SYS>>
You are an advanced Python coding assistant. Your primary task is to write Python code to solve problems based on the given arithmetic problem and a similar problem's solution approach.

The input will include:
1. A problem written in English that explains a scenario requiring arithmetic calculations.
2. A solution plan for a similar problem, which you should reference while creating the code.

Your output must:
- Generate clean, efficient, and accurate Python code to solve the problem.
- Use the provided similar problem solution as a guide to structure your code.
- Ensure that the code is well-commented and follows Python best practices.
- Provide only Python code as the output, without any additional explanations or text.
<</SYS>>
Problem:
{original_question}

Solution Plan for a Similar Problem:
{plan}

Write Python code to solve the problem based on the above inputs.
[/INST]
"""

    """
    [INST] <<SYS>>Your task is to generate Python code to solve a given problem.<</SYS>>
Below is a Python code that requires implementation. Please complete the parts indicated by comments.

Initial code and sections that need implementation:
{original_question}

Implementation plan:
{plan}

Based on the above information, complete the commented sections.
[/INST]
"""


# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# Your task is to generate Python code to solve a given problem.
# To help you, we provide the original problem and the solution approach to a similar problem.
# Use the similar problem's solution as a reference to guide your implementation.
# <|eot_id|>
# <|start_header_id|>user<|end_header_id|>
# Original Problem:
# {original_question}

# Solution Approach for a Similar Problem (Reference):
# {plan}

# Instructions:
# - Write Python code to solve the original problem.
# - Refer to the solution approach of the similar problem to structure and guide your code.
# - Ensure the code is clean, efficient, and well-commented.
# - The final output of your code should solve the original problem.

# Provide only the Python code as the output.
# <|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>
# """
#     full_prompt = f"""
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# You are an expert coding assistant.<|eot_id|>
# <|start_header_id|>user<|end_header_id|>
# {original_question}
# {plan}

# Provide only the Python code as the output.
# <|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>
# """

#     full_prompt = f"""
# [INST] <<SYS>>{system_prompt}<</SYS>>
# # Below is a Python code that requires implementation. Please complete the parts indicated by comments.

# # Initial code and sections that need implementation:
# {original_question}

# # Implementation plan:
# {plan}

# # Based on the above information, complete the commented sections.
# [/INST]
# """

#     """

# ## Problem to be solved :
# {original_question}
# ## Planning :
# {plan}

#     Given a competitive programming problem generate Python3 code to solve the problem.


# ## Let's think step by step.
# ------------
# Important:
# ## Your response must contain only the Python code to solve this problem. Do not add extra explanation or words.
# """
    return full_prompt

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

# Execute pipeline

In [14]:
from tqdm import tqdm
output_file = "generated_code-withAgentPrompt.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["question", "answer", "plan", "code"])  # CSV 헤더 작성

    # for idx, data in enumerate(dataset["test"]):
    for idx, data in enumerate(tqdm(dataset["test"].select(range(1001)), desc="Processing test data")):
        question = data["question"]
        answer = data["answer"]

        similar_question, similar_snippet, distance = RAG_AGENT(question)
        # print(similar_question, similar_snippet)

        plan = PLANNING_AGENT(question, similar_question, similar_snippet, distance)
        # plan = data["plan"]
        # print(plan)
        # break

        # print(data_prompt)
        # break

        prompt_for_generate_code = generate_and_tokenize_prompt_for_generate_code(question, similar_question, similar_snippet, distance, plan)
        # print(prompt_for_generate_code)
        # break

        # 입력 프롬프트를 토크나이즈하여 길이 계산
        # input_ids = code_tokenizer.encode(prompt_for_generate_code, return_tensors='pt', truncation=True, max_length=1024).to('cuda')
        input_ids = code_tokenizer.encode(prompt_for_generate_code, return_tensors='pt', truncation=True).to('cuda')
        input_length = input_ids.shape[-1]

        # max_new_tokens를 입력 프롬프트 길이 + 100으로 설정
        # max_new_tokens = input_length + 900

        # print(prompt_for_generate_code)
        # break

        sequences = code_pipeline(
            prompt_for_generate_code,
            # question,
            # data_prompt,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
            num_return_sequences=1,
            eos_token_id=code_tokenizer.eos_token_id,
            pad_token_id=code_tokenizer.eos_token_id,
            repetition_penalty=1.2,
            # max_new_tokens=max_new_tokens,
            max_new_tokens=512,
            # max_length=500,
        )

        generated_code = sequences[0]['generated_text']

        keyword = "[/INST]"
        if keyword in generated_code:
            generated_code = generated_code.split(keyword)[1].strip()

        # keyword = "<|eot_id|>"
        # if keyword in generated_code:
        #     generated_code = generated_code.split(keyword)[0].strip()

        # print(prompt_for_generate_code)
        # print('--------------------------')
        # print(sequences[0]['generated_text'])
        # print(generated_code)
        # print('=====================================')
        # break
        writer.writerow([question, answer, plan, generated_code])
        # writer.writerow([task_id, data_prompt.replace('\n', '\\n'), generated_code.replace('\n', '\\n')])
        # break

Processing test data: 100%|██████████| 1001/1001 [4:14:08<00:00, 15.23s/it]


In [16]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

In [17]:
# # !cp generated_code-agent.csv /content/drive/MyDrive/

import pandas as pd

df = pd.read_csv(output_file)

from datasets import Dataset
from google.colab import userdata

dataset = Dataset.from_pandas(df)
dataset.push_to_hub("donghuna/generated_code-gsm8k-Agent-A", token=userdata.get('HUG_TOKEN'))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/donghuna/generated_code-gsm8k-Agent-A/commit/b2d3afc3e2a77de2e1f914d15b8aa22ee0b98208', commit_message='Upload dataset', commit_description='', oid='b2d3afc3e2a77de2e1f914d15b8aa22ee0b98208', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/donghuna/generated_code-gsm8k-Agent-A', endpoint='https://huggingface.co', repo_type='dataset', repo_id='donghuna/generated_code-gsm8k-Agent-A'), pr_revision=None, pr_num=None)